In [33]:
import numpy as np
import pandas as pd

import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

In [34]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

def vsmape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * smap

# 데이터 확인

In [35]:
census = pd.read_csv('./data/census_starter.csv')
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sub = pd.read_csv('./data/sample_submission.csv')
print(train.shape, test.shape, sub.shape)

(122265, 7) (25080, 3) (25080, 2)


In [36]:
train.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 122265 non-null  object 
 1   cfips                  122265 non-null  int64  
 2   county                 122265 non-null  object 
 3   state                  122265 non-null  object 
 4   first_day_of_month     122265 non-null  object 
 5   microbusiness_density  122265 non-null  float64
 6   active                 122265 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 6.5+ MB


In [38]:
train.describe()

,cfips,microbusiness_density,active
count,122265.000000,122265.000000,1.222650e+05
mean,30376.037640,3.817671,6.442858e+03
std,15143.508721,4.991087,3.304001e+04
min,1001.000000,0.000000,0.000000e+00
25%,18177.000000,1.639344,1.450000e+02
50%,29173.000000,2.586543,4.880000e+02
75%,45077.000000,4.519231,2.124000e+03
max,56045.000000,284.340030,1.167744e+06


In [39]:
train["cfips"].nunique()

3135

In [40]:
test.head()

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01


In [41]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25080 entries, 0 to 25079
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              25080 non-null  object
 1   cfips               25080 non-null  int64 
 2   first_day_of_month  25080 non-null  object
dtypes: int64(1), object(2)
memory usage: 587.9+ KB


In [42]:
census.head()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


In [43]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pct_bb_2017            3142 non-null   float64
 1   pct_bb_2018            3142 non-null   float64
 2   pct_bb_2019            3142 non-null   float64
 3   pct_bb_2020            3141 non-null   float64
 4   pct_bb_2021            3141 non-null   float64
 5   cfips                  3142 non-null   int64  
 6   pct_college_2017       3142 non-null   float64
 7   pct_college_2018       3142 non-null   float64
 8   pct_college_2019       3142 non-null   float64
 9   pct_college_2020       3141 non-null   float64
 10  pct_college_2021       3141 non-null   float64
 11  pct_foreign_born_2017  3142 non-null   float64
 12  pct_foreign_born_2018  3142 non-null   float64
 13  pct_foreign_born_2019  3142 non-null   float64
 14  pct_foreign_born_2020  3141 non-null   float64
 15  pct_

In [44]:
census["cfips"].nunique()

3142

In [45]:
sub.head()

,row_id,microbusiness_density
0,1001_2022-11-01,3.817671
1,1003_2022-11-01,3.817671
2,1005_2022-11-01,3.817671
3,1007_2022-11-01,3.817671
4,1009_2022-11-01,3.817671


In [46]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25080 entries, 0 to 25079
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   row_id                 25080 non-null  object 
 1   microbusiness_density  25080 non-null  float64
dtypes: float64(1), object(1)
memory usage: 392.0+ KB


# 데이터 전처리 & 분석

In [47]:
train.corr()

,cfips,microbusiness_density,active
cfips,1.000000,-0.011767,-0.062760
microbusiness_density,-0.011767,1.000000,0.316981
active,-0.062760,0.316981,1.000000


In [ ]:
plt.figure()
sns.heatmap(train.corr(), annot = True)
plt.show()

In [ ]:
plt.figure()
sns.pairplot(train)
plt.show()

In [ ]:
plt.figure()
sns.pairplot(census)
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
sns.heatmap(census.corr(), annot = True)
plt.show()

In [52]:
train['istest'] = 0
test['istest'] = 1
raw = pd.concat([train, test]).sort_values(['cfips','row_id']).reset_index(drop=True)

- istest : train, test 구분

In [56]:
raw['first_day_of_month'] = pd.to_datetime(raw["first_day_of_month"])
raw['county'] = raw.groupby('cfips')['county'].ffill()
raw['state'] = raw.groupby('cfips')['state'].ffill()
raw["year"] = raw["first_day_of_month"].dt.year
raw["month"] = raw["first_day_of_month"].dt.month
raw["dcount"] = raw.groupby(['cfips'])['row_id'].cumcount()
raw['county_i'] = raw["cfips"] % 1000
raw['state_i'] = raw["cfips"] // 1000

- train의 cfips를 이용해 test 부분의 state, county 채움
- dcount : cfips별 row_id로 time step 생성
- cfips 앞 2자리는 county, 뒤 세자리는 state

In [57]:
raw.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,year,month,dcount,county_i,state_i
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249.0,0,2019,8,0,1,1
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,2019,9,1,1,1
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,2019,10,2,1,1
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,2019,11,3,1,1
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,2019,12,4,1,1


In [58]:
raw["county_i"].value_counts()

1      2303
5      2303
3      2303
9      2256
13     2209
       ... 
369      47
371      47
373      47
375      47
78       47
Name: county_i, Length: 319, dtype: int64

In [59]:
raw.tail()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,year,month,dcount,county_i,state_i
147340,56045_2023-02-01,56045,Weston County,Wyoming,2023-02-01,NaN,NaN,1,2023,2,42,45,56
147341,56045_2023-03-01,56045,Weston County,Wyoming,2023-03-01,NaN,NaN,1,2023,3,43,45,56
147342,56045_2023-04-01,56045,Weston County,Wyoming,2023-04-01,NaN,NaN,1,2023,4,44,45,56
147343,56045_2023-05-01,56045,Weston County,Wyoming,2023-05-01,NaN,NaN,1,2023,5,45,45,56
147344,56045_2023-06-01,56045,Weston County,Wyoming,2023-06-01,NaN,NaN,1,2023,6,46,45,56


In [60]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147345 entries, 0 to 147344
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   row_id                 147345 non-null  object        
 1   cfips                  147345 non-null  int64         
 2   county                 147345 non-null  object        
 3   state                  147345 non-null  object        
 4   first_day_of_month     147345 non-null  datetime64[ns]
 5   microbusiness_density  122265 non-null  float64       
 6   active                 122265 non-null  float64       
 7   istest                 147345 non-null  int64         
 8   year                   147345 non-null  int64         
 9   month                  147345 non-null  int64         
 10  dcount                 147345 non-null  int64         
 11  county_i               147345 non-null  int64         
 12  state_i                147345 non-null  int6

In [61]:
raw.corr()

,cfips,microbusiness_density,active,istest,year,month,dcount,county_i,state_i
cfips,1.000000e+00,-0.011767,-0.062760,2.784332e-15,5.977495e-12,-8.470577e-15,-7.862261e-15,1.691223e-01,9.999759e-01
microbusiness_density,-1.176709e-02,1.000000,0.316981,NaN,1.702471e-02,2.121343e-03,1.862944e-02,-1.702893e-02,-1.166093e-02
active,-6.275991e-02,0.316981,1.000000,NaN,4.035053e-03,4.860858e-04,4.410361e-03,-4.073179e-02,-6.254637e-02
istest,2.784332e-15,NaN,NaN,1.000000e+00,6.433189e-01,-1.284741e-01,6.510860e-01,7.195322e-16,2.787015e-15
year,5.977495e-12,0.017025,0.004035,6.433189e-01,1.000000e+00,-3.602007e-01,9.708063e-01,1.153995e-13,5.983552e-12
month,-8.470577e-15,0.002121,0.000486,-1.284741e-01,-3.602007e-01,1.000000e+00,-1.259213e-01,5.702012e-16,-8.485555e-15
dcount,-7.862261e-15,0.018629,0.004410,6.510860e-01,9.708063e-01,-1.259213e-01,1.000000e+00,-4.191241e-15,-7.838706e-15
county_i,1.691223e-01,-0.017029,-0.040732,7.195322e-16,1.153995e-13,5.702012e-16,-4.191241e-15,1.000000e+00,1.622795e-01
state_i,9.999759e-01,-0.011661,-0.062546,2.787015e-15,5.983552e-12,-8.485555e-15,-7.838706e-15,1.622795e-01,1.000000e+00


In [ ]:
plt.figure()
sns.pairplot(raw)
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
sns.heatmap(raw.corr(), annot = True)
plt.show()

In [ ]:
lag = 1
raw[f'mbd_lag_{lag}'] = raw.groupby('cfips')['microbusiness_density'].shift(lag).bfill()
raw['dif'] = (raw['microbusiness_density'] / raw[f'mbd_lag_{lag}']).fillna(1).clip(0, None) - 1
raw.loc[(raw[f'mbd_lag_{lag}']==0), 'dif'] = 0
raw.loc[(raw[f'microbusiness_density']>0) & (raw[f'mbd_lag_{lag}']==0), 'dif'] = 1
raw['dif'] = raw['dif'].abs()
raw.groupby('dcount')['dif'].sum().plot()

- mbd_lag_ : microbusiness_density 지연값
- dif : microbusiness_density, 지연값 차 크기

In [65]:
raw.groupby('dcount')['dif'].sum().sort_values()

dcount
0       0.000000
44      0.000000
43      0.000000
42      0.000000
41      0.000000
40      0.000000
39      0.000000
45      0.000000
46      0.000000
33     28.565758
23     33.674223
22     33.709702
24     35.103748
28     35.152939
26     36.942643
37     37.078203
30     38.996456
21     40.255331
32     40.989782
38     41.124939
4      41.783860
25     42.336087
36     42.439082
14     43.089840
19     46.119200
20     46.370614
16     46.601331
15     47.870846
34     48.424032
10     48.704668
13     50.157551
31     52.718056
29     52.722447
12     53.840883
27     54.288004
11     58.304183
5      60.688540
17     67.068410
9      69.020624
7      71.639301
35     75.967980
3      84.472938
8      88.890771
6      97.568436
2     109.819823
1     164.301307
18    282.215873
Name: dif, dtype: float64

- dcount 39이상부터 0임을 확인

In [ ]:
raw0 = raw.copy()
for lag in range(1, 7):
    raw0[f'mbd_lag_{lag}'] = raw0.groupby('cfips')['microbusiness_density'].shift(lag).bfill()
    raw0['dif'] = (raw0['microbusiness_density'] / raw0[f'mbd_lag_{lag}']).fillna(1).clip(0, None) - 1
    raw0.loc[(raw0[f'mbd_lag_{lag}']==0), 'dif'] = 0
    raw0.loc[(raw0[f'microbusiness_density']>0) & (raw0[f'mbd_lag_{lag}']==0), 'dif'] = 1
    raw0['dif'] = raw0['dif'].abs()
    raw0.groupby('dcount')['dif'].sum().plot()

In [69]:
outliers = []
cnt = 0
for j in tqdm(raw["cfips"].unique()):
    indices = (raw['cfips']==j)
    tmp = raw.loc[indices].copy().reset_index(drop=True)
    var = tmp["microbusiness_density"].values.copy()
    
    for i in range(37, 2, -1):
        thr = 0.20*np.mean(var[:i])
        difa = abs(var[i]-var[i-1])
        if (difa>=thr):
            var[:i] *= (var[i]/var[i-1])
            outliers.append(j)
            cnt+=1
    var[0] = var[1]*0.99
    raw.loc[indices, 'microbusiness_density'] = var
    
outliers = np.unique(outliers)
len(outliers), cnt

  0%|          | 0/3135 [00:00<?, ?it/s]

C:\Users\gmrfy\AppData\Local\Temp\ipykernel_25512\4202392711.py:12: RuntimeWarning: invalid value encountered in double_scalars
  var[:i] *= (var[i]/var[i-1])
C:\Users\gmrfy\AppData\Local\Temp\ipykernel_25512\4202392711.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  var[:i] *= (var[i]/var[i-1])
C:\Users\gmrfy\AppData\Local\Temp\ipykernel_25512\4202392711.py:12: RuntimeWarning: invalid value encountered in multiply
  var[:i] *= (var[i]/var[i-1])
C:\Users\gmrfy\AppData\Local\Temp\ipykernel_25512\4202392711.py:10: RuntimeWarning: invalid value encountered in double_scalars
  difa = abs(var[i]-var[i-1])


(481, 732)

- outliers : 

In [70]:
outliers

array([ 1013,  1035,  1037,  1045,  1057,  1059,  1085,  1111,  1125,
        1131,  1133,  2060,  2070,  2100,  2164,  2188,  2282,  5011,
        5029,  5031,  5049,  5061,  5065,  5077,  5079,  5081,  5091,
        5093,  5113,  5141,  6005,  6015,  8011,  8014,  8031,  8047,
        8055,  8057,  8069,  8079,  8081,  8105,  8121, 10003, 10005,
       12001, 12013, 12029, 12037, 12045, 12051, 12065, 12077, 12107,
       12131, 13007, 13019, 13025, 13033, 13037, 13061, 13087, 13101,
       13119, 13131, 13149, 13169, 13193, 13239, 13243, 13251, 13287,
       13291, 13301, 13307, 13317, 15005, 16009, 16021, 16025, 16033,
       16037, 16045, 16049, 16051, 16077, 16079, 17013, 17037, 17063,
       17065, 17075, 17105, 17109, 17127, 17147, 17175, 17191, 18017,
       18025, 18031, 18039, 18041, 18049, 18073, 18083, 18087, 18103,
       18143, 18145, 18153, 18171, 19001, 19003, 19005, 19033, 19037,
       19051, 19063, 19065, 19067, 19071, 19125, 19129, 19133, 19143,
       19165, 19175,

In [ ]:
lag = 1
raw[f'mbd_lag_{lag}'] = raw.groupby('cfips')['microbusiness_density'].shift(lag).bfill()
raw['dif'] = (raw['microbusiness_density'] / raw[f'mbd_lag_{lag}']).fillna(1).clip(0, None) - 1
raw.loc[(raw[f'mbd_lag_{lag}']==0), 'dif'] = 0
raw.loc[(raw[f'microbusiness_density']>0) & (raw[f'mbd_lag_{lag}']==0), 'dif'] = 1
raw['dif'] = raw['dif'].abs()
raw.groupby('dcount')['dif'].sum().plot()

In [ ]:
raw.groupby('dcount')['dif'].sum().sort_values()

In [ ]:
raw0 = raw.copy()
for lag in range(1, 13):
    raw0[f'mbd_lag_{lag}'] = raw0.groupby('cfips')['microbusiness_density'].shift(lag).bfill()
    raw0['dif'] = (raw0['microbusiness_density'] / raw0[f'mbd_lag_{lag}']).fillna(1).clip(0, None) - 1
    raw0.loc[(raw0[f'mbd_lag_{lag}']==0), 'dif'] = 0
    raw0.loc[(raw0[f'microbusiness_density']>0) & (raw0[f'mbd_lag_{lag}']==0), 'dif'] = 1
    raw0['dif'] = raw0['dif'].abs()
    raw0.groupby('dcount')['dif'].sum().plot()

In [43]:
raw_cfips_col = raw["cfips"].unique().tolist()

In [ ]:
for i in raw_cfips_col:
    plt.figure()
    raw.loc[raw["cfips"] == i].plot(x='dcount', y='microbusiness_density')
    plt.title(f"cfips : {i}")
    plt.show()

In [ ]:
raw.loc[raw["cfips"] == 1013].plot(x='dcount', y='microbusiness_density')
raw.loc[raw["cfips"] == 21215].plot(x='dcount', y='microbusiness_density')

In [ ]:
raw.loc[raw["cfips"] == 28055].plot(x='dcount', y='microbusiness_density')
raw.loc[raw["cfips"] == 48269].plot(x='dcount', y='microbusiness_density')

In [ ]:
raw['target'] = raw.groupby('cfips')['microbusiness_density'].shift(-1)
raw['target'] = raw['target']/raw['microbusiness_density'] - 1

raw.loc[raw['cfips']==28055, 'target'] = 0.0
raw.loc[raw['cfips']==48269, 'target'] = 0.0

raw.iloc[-20:,:20]

- target : 다음 달 / 이번 달 - 1 -> 이번달 대비 다음달에 증가한 비율

In [ ]:
raw.loc[raw['dcount']==38]

In [ ]:
raw['target'].clip(-0.2, 0.2).hist(bins=100)

In [ ]:
raw['target'].clip(-0.05, 0.05).hist(bins=100)

In [10]:
raw['lastactive'] = raw.groupby('cfips')['active'].transform('last')
raw['lastactive'].head()

0    1472.0
1    1472.0
2    1472.0
3    1472.0
4    1472.0
Name: lastactive, dtype: float64

- lastactive : cfips 별 가장 최근 active

In [ ]:
dt = raw.loc[raw["dcount"]==28].groupby('cfips')['microbusiness_density'].agg('last')
raw['lasttarget'] = raw['cfips'].map(dt)

raw['lastactive'].clip(0, 10000).hist(bins=30)

In [12]:
def build_features(raw, target='microbusiness_density', target_act='active', lags = 6):
    feats = []
    for lag in range(1, lags):
        raw[f'mbd_lag_{lag}'] = raw.groupby('cfips')[target].shift(lag)
        raw[f'act_lag_{lag}'] = raw.groupby('cfips')[target_act].diff(lag)
        feats.append(f'mbd_lag_{lag}')
        feats.append(f'act_lag_{lag}')
        
    lag = 1
    for window in [2, 4, 6, 8, 10]:
        raw[f'mbd_rollmea{window}_{lag}'] = raw.groupby('cfips')[f'mbd_lag_{lag}'].transform(lambda x: x.rolling(window, min_periods=1).sum())        
        raw[f'mbd_rollmea{window}_{lag}'] = raw[f'mbd_lag_{lag}'] - raw[f'mbd_rollmea{window}_{lag}']
        feats.append(f'mbd_rollmea{window}_{lag}')
    
    census_columns = list(census.columns)
    census_columns.remove("cfips")
    
    raw = raw.merge(census, on="cfips", how="left")
    feats += census_columns
    
    return raw, feats

- 생성할 지연값의 개수와 shift 크기 지정 및 census merge하는 함수
    - mbd_lag_ : target의 지연값
    - act_lag_ : 현재 target_act 값과 이전 1,2, .. lags-1 개월 target_act 값의 차
    - mbd_rollmea : 이전의 2, 4, 6, 8, 10월의 target 값의 합과 mbd_lag_의 차

In [13]:
raw, feats = build_features(raw, 'target', 'active', lags = 6)
features = ['state_i']
features += feats
print(features)

['state_i', 'mbd_lag_1', 'act_lag_1', 'mbd_lag_2', 'act_lag_2', 'mbd_lag_3', 'act_lag_3', 'mbd_lag_4', 'act_lag_4', 'mbd_lag_5', 'act_lag_5', 'mbd_rollmea2_1', 'mbd_rollmea4_1', 'mbd_rollmea6_1', 'mbd_rollmea8_1', 'mbd_rollmea10_1', 'pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017', 'pct_college_2018', 'pct_college_2019', 'pct_college_2020', 'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018', 'pct_foreign_born_2019', 'pct_foreign_born_2020', 'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018', 'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021', 'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019', 'median_hh_inc_2020', 'median_hh_inc_2021']


In [59]:
raw.loc[raw.dcount==38, features].head(10)

,state_i,mbd_lag_1,act_lag_1,mbd_lag_2,act_lag_2,mbd_lag_3,act_lag_3,mbd_lag_4,act_lag_4,mbd_lag_5,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
38,1,0.006152,9.0,0.005498,17.0,-0.004107,11.0,0.027426,50.0,0.009943,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
85,1,0.002170,31.0,-0.017601,-225.0,-0.009601,-366.0,0.007754,-253.0,0.075895,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
132,1,0.020921,5.0,0.008439,7.0,-0.016597,3.0,0.021187,8.0,0.004255,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
179,1,-0.021367,-5.0,0.017391,-1.0,-0.025424,-7.0,0.039647,2.0,0.022523,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
226,1,-0.010949,-9.0,0.008589,-2.0,0.002460,0.0,0.033037,26.0,0.014175,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0
273,1,0.012987,1.0,0.013158,2.0,0.013333,3.0,0.013513,4.0,0.027778,...,0.4,0.3,0.5,0.3,0.2,29655,32152.0,37785,33866.0,29063.0
320,1,0.009036,3.0,0.027864,12.0,0.031949,22.0,0.029605,31.0,0.097473,...,1.1,1.4,1.7,1.3,1.4,36326,39109.0,40688,44850.0,45236.0
367,1,-0.003180,-8.0,0.007609,11.0,0.002006,16.0,0.021311,68.0,0.012448,...,1.4,1.4,1.2,1.0,1.0,43686,45197.0,47255,50128.0,50977.0
414,1,-0.040201,-16.0,-0.012407,-21.0,-0.017073,-28.0,0.053985,-7.0,-0.012690,...,2.4,2.1,2.1,2.3,1.8,37342,39872.0,42289,43875.0,47232.0
461,1,-0.012605,-3.0,0.000000,-3.0,-0.008333,-5.0,0.021277,0.0,-0.103054,...,1.4,1.3,1.2,0.9,0.4,40041,41014.0,41919,42509.0,43475.0


In [ ]:
raw['lasttarget'].clip(0,10).hist(bins=100)

# 모델

In [22]:
def model3():
    from sklearn.ensemble import VotingRegressor, StackingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
    import lightgbm as lgb
    import xgboost as xgb
    import catboost as cat
    from sklearn.pipeline import Pipeline
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import KNNImputer, IterativeImputer
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.linear_model import LinearRegression, BayesianRidge
    from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor

    params = {
    'n_estimators': 200,
    'verbosity': -1,
    'objective': 'l1',
    'random_state': 42,
    'colsample_bytree': 0.8841279649367693,
    'colsample_bynode': 0.10142964450634374,
    'max_depth': 8,
    'learning_rate': 0.013647749926797374,
    'reg_alpha': 1.8386216853616875,
    'reg_lambda': 7.557660410418351,
    'num_leaves': 61,
    'min_child_samples': 213}

    lgb_model = lgb.LGBMRegressor(**params)

    
    xgb_model = xgb.XGBRegressor(
        objective='reg:pseudohubererror',
        tree_method="hist",
        n_estimators=795,
        learning_rate=0.0075,
        max_leaves = 17,
        subsample=0.50,
        colsample_bytree=0.50,
        max_bin=4096,
        n_jobs=2
    )
    
    cat_model = cat.CatBoostRegressor(
        iterations=1200,
        loss_function="MAPE",
        verbose=0,
        learning_rate=0.075,
        l2_leaf_reg=0.2,
        subsample=0.50,
        max_bin=4096
    )
    
    hgr_model = HistGradientBoostingRegressor()

    ets_model = Pipeline([
        ('imputer',  IterativeImputer(max_iter = 20, n_nearest_features=2)),
        ('ets', ExtraTreesRegressor())
    ])

    ba_model = Pipeline([
        ('imputer',  IterativeImputer(max_iter = 20, n_nearest_features=2)),
        ('BayesianRidge', BayesianRidge())
    ])
    
    et_model = Pipeline([
        ('imputer',  IterativeImputer(n_nearest_features=2)),
        ('et', ExtraTreeRegressor())
    ])
    
    rfr_model = Pipeline([
        ('imputer',  IterativeImputer(max_iter = 20, n_nearest_features=2)),
        ('rfr', RandomForestRegressor())
    ])
    
    dtr_model = Pipeline([
        ('imputer',  IterativeImputer(n_nearest_features=2)),
        ('dtr', DecisionTreeRegressor())
    ])
    
    knn_model = Pipeline([
        ('imputer',  KNNImputer(n_neighbors=2)),
        ('knn', KNeighborsRegressor(5))
    ])
    
    stack_model = StackingRegressor([
        ('knn', knn_model),
        ('hgr', hgr_model),
        ('rfr', rfr_model),
        ('ets',ets_model),
        ('et',et_model),
        ('dtr',dtr_model),
        ('BayesianRidge',ba_model)])
        
    return VotingRegressor ([('xgb', xgb_model),
                            ('lgb', lgb_model),
                            ('cat', cat_model),
                             ('stack', stack_model),
                            ])

In [5]:
blacklist = []
blacklistcfips = []
ACT_THR = 140
ABS_THR = 0
raw['ypred_last'] = np.nan
raw['ypred'] = np.nan
raw['k'] = 1.
VAL = []

In [6]:
xgbm = xgb.XGBRegressor(
        objective='reg:pseudohubererror',
        tree_method="hist",
        n_estimators=795,
        learning_rate=0.0075,
        max_leaves = 17,
        subsample=0.50,
        colsample_bytree=0.50,
        max_bin=4096,
        n_jobs=2
    )

In [17]:
for TS in range(29, 38):
    print(TS)
    
    model = xgbm
    
    # 훈련용 데이터
    train_indices = (raw["istest"]==0) & (raw["dcount"] < TS) & (raw["dcount"] >= 1) & (raw["lastactive"]>ACT_THR) & (raw["lasttarget"]>ABS_THR)
    # 검증용 데이터
    valid_indices = (raw["istest"]==0) & (raw["dcount"] == TS)
    model.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, 'target'].clip(-0.0043, 0.0045)
    )

    ypred = model.predict(raw.loc[valid_indices, features])
    raw.loc[valid_indices, 'k'] = ypred + 1
    raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

    # 검증
    lastval = raw.loc[raw["dcount"]==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
    dt = raw.loc[raw["dcount"]==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']
    
    df = raw.loc[raw["dcount"]==(TS+1), ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
    df['pred'] = df['cfips'].map(dt)
    df['lastval'] = df['cfips'].map(lastval)
    
    df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
    df.loc[df['lastval']<=ABS_THR, 'pred'] = df.loc[df['lastval']<=ABS_THR, 'lastval']
    df.loc[df['state'].isin(blacklist), 'pred'] = df.loc[df['state'].isin(blacklist), 'lastval']
    df.loc[df['cfips'].isin(blacklistcfips), 'pred'] = df.loc[df['cfips'].isin(blacklistcfips), 'lastval']
    raw.loc[raw["dcount"]==(TS+1), 'ypred'] = df['pred'].values
    raw.loc[raw["dcount"]==(TS+1), 'ypred_last'] = df['lastval'].values
    
    print(f'TS: {TS}')
    print('Last Value SMAPE:', smape(df['microbusiness_density'], df['lastval']) )
    print('SMAPE:', smape(df['microbusiness_density'], df['pred']))
    print()

29
TS: 29
Last Value SMAPE: 1.0868726017655663
SMAPE: 1.0768312859654576

30
TS: 30
Last Value SMAPE: 1.318087470449913
SMAPE: 1.249836902283442

31
TS: 31
Last Value SMAPE: 1.1258309832479911
SMAPE: 1.1036775437619872

32
TS: 32
Last Value SMAPE: 0.897969439640235
SMAPE: 0.9786975547095251

33
TS: 33
Last Value SMAPE: 1.3686285670946152
SMAPE: 1.3313452310059841

34
TS: 34
Last Value SMAPE: 2.2033066808448543
SMAPE: 2.0826018540015023

35
TS: 35
Last Value SMAPE: 1.2797936949214384
SMAPE: 1.3715621735471746

36
TS: 36
Last Value SMAPE: 1.034314865882525
SMAPE: 1.014226262900631

37
TS: 37
Last Value SMAPE: 1.101119095956366
SMAPE: 1.0650666635908306



In [18]:
ind = (raw["dcount"]>=30)&(raw["dcount"]<=38)
print('SMAPE:', smape( raw.loc[ind, 'microbusiness_density'], raw.loc[ind, 'ypred']))
print('Last Value SMAPE:', smape(raw.loc[ind, 'microbusiness_density'], raw.loc[ind, 'ypred_last']))

SMAPE: 1.2526494968629482
Last Value SMAPE: 1.2684359333115005


In [19]:
raw['error'] = vsmape(raw['microbusiness_density'], raw['ypred'])
raw['error_last'] = vsmape(raw['microbusiness_density'], raw['ypred_last'])
raw.loc[(raw.dcount==30), ['microbusiness_density', 'ypred', 'error', 'error_last'] ]

,microbusiness_density,ypred,error,error_last
30,3.334431,3.302485,0.962688,1.135557
77,7.823300,7.752988,0.902803,1.155810
124,1.206827,1.188813,1.503903,1.687769
171,1.236650,1.216129,1.673294,1.834867
218,1.777708,1.754625,1.306961,1.403959
...,...,...,...,...
147140,2.892446,2.929649,1.277989,1.179620
147187,25.438322,25.572762,0.527099,0.368550
147234,3.954258,3.758092,5.087051,5.183206
147281,3.027295,3.031646,0.143594,0.000000


In [20]:
dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38)].groupby(['state'])['error', 'error_last'].mean()
dt['hit'] = dt['error'] - dt['error_last']
dt = dt.sort_values('hit', ascending=True)
dt.shape

C:\Users\gmrfy\AppData\Local\Temp\ipykernel_7200\2697617936.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38) ].groupby(['state'])['error', 'error_last'].mean()


(51, 3)

In [21]:
dt

,error,error_last,hit
state,,,
Delaware,1.820101,1.964820,-0.144719
District of Columbia,0.423252,0.503024,-0.079772
Florida,1.047340,1.097365,-0.050025
New Jersey,0.659228,0.703606,-0.044377
Maine,0.969204,1.011834,-0.042630
New Hampshire,0.616725,0.658417,-0.041692
Connecticut,0.696042,0.735432,-0.039389
South Carolina,1.116399,1.154938,-0.038538
Utah,1.118050,1.156125,-0.038075


In [36]:
dt.index[-8:]

Index(['Hawaii', 'Wyoming', 'Nebraska', 'New Mexico', 'North Dakota', 'Alaska',
       'Rhode Island', 'Vermont'],
      dtype='object', name='state')

In [23]:
dt.index[-10:]

Index(['South Dakota', 'Iowa', 'Hawaii', 'Wyoming', 'Nebraska', 'New Mexico',
       'North Dakota', 'Alaska', 'Rhode Island', 'Vermont'],
      dtype='object', name='state')

In [24]:
dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38) ].groupby(['cfips','dcount'])['error', 'error_last'].last()
dt['miss'] = dt['error'] > dt['error_last']
dt = dt.groupby('cfips')['miss'].mean()
dt = dt.loc[dt>=0.50]
dt.shape

C:\Users\gmrfy\AppData\Local\Temp\ipykernel_7200\956906285.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38) ].groupby(['cfips','dcount'])['error', 'error_last'].last()


(814,)

In [25]:
len(dt.index)

814

In [ ]:
','.join([str(i) for i in dt.index])

In [ ]:
for d in dt.index[:10]:
    raw.loc[raw.cfips==d].plot(x='dcount', y=['microbusiness_density', 'ypred'], title=str(d))

In [ ]:
raw.iloc[-40:, :16]

In [17]:
blacklist = [
     'South Dakota', 'Iowa', 'Hawaii', 'Wyoming', 'Nebraska', 'New Mexico',
       'North Dakota', 'Alaska', 'Rhode Island', 'Vermont'
]
blacklistcfips = [
    1019,1027,1029,1039,1045,1049,1067,1077,1091,1099,1101,1123,1133,2110,2122,2150,2198,2220,4001,4009,4012,4021,4023,5001,5003,5005,5019,5027,5029,5031,5035,5047,5057,5059,5063,5065,5071,5083,5087,5091,5093,5097,5107,5115,5121,5131,5137,5141,6005,6015,6027,6033,6037,6055,6063,6069,6071,6081,6093,6095,6097,6103,6105,6115,8003,8007,8015,8019,8027,8047,8049,8051,8055,8059,8065,8071,8075,8085,8091,8099,8103,8105,8109,8117,9009,9015,12007,12009,12017,12019,12029,12047,12055,12065,12075,12099,12127,13009,13015,13017,13019,13047,13055,13071,13075,13081,13083,13091,13107,13115,13117,13119,13121,13135,13143,13147,13181,13187,13199,13219,13221,13225,13229,13245,13247,13257,13279,13281,13293,13319,15007,16001,16005,16009,16013,16015,16031,16035,16037,16041,16043,16049,16059,16067,16077,17007,17011,17015,17025,17031,17033,17045,17051,17063,17067,17077,17085,17089,17099,17103,17105,17107,17115,17117,17123,17127,17137,17141,17143,17145,17147,17149,17163,17173,17177,17179,17181,17187,18001,18007,18015,18019,18037,18039,18041,18051,18061,18065,18067,18075,18079,18083,18087,18093,18099,18103,18111,18117,18123,18137,18139,18145,18153,18159,18177,19005,19007,19011,19019,19031,19033,19037,19041,19047,19051,19053,19055,19059,19061,19069,19073,19077,19081,19089,19095,19099,19105,19109,19123,19139,19141,19151,19157,19161,19165,19171,19175,19179,19181,19195,20009,20011,20035,20037,20079,20099,20107,20113,20121,20123,20125,20127,20139,20151,20157,20159,20161,20191,20193,21001,21003,21013,21017,21029,21045,21047,21059,21065,21085,21089,21093,21097,21099,21101,21103,21115,21121,21125,21137,21141,21151,21155,21157,21161,21179,21191,21193,21197,21199,21215,21217,21227,21239,22007,22015,22019,22031,22039,22041,22047,22053,22069,22085,22087,22089,22109,22111,22115,22119,23003,23021,23027,23029,24011,24015,24031,24037,24039,25011,25015,26001,26003,26007,26011,26019,26021,26023,26025,26027,26037,26041,26043,26045,26051,26053,26057,26059,26061,26065,26079,26095,26097,26101,26103,26105,26109,26113,26115,26117,26119,26129,26135,26137,26141,26143,26155,26161,26163,27005,27013,27015,27021,27025,27047,27055,27057,27077,27079,27095,27099,27103,27105,27109,27113,27117,27129,27131,27135,27159,27167,27169,28017,28023,28025,28035,28045,28049,28079,28099,28115,28137,28141,28147,28159,29001,29013,29015,29019,29031,29049,29051,29055,29057,29059,29063,29065,29069,29071,29083,29101,29123,29135,29139,29143,29147,29153,29159,29175,29177,29183,29195,29207,29209,29213,29215,29217,29229,30021,30027,30035,30041,30045,30049,30053,30057,30061,30067,30083,30085,30089,30093,31001,31011,31013,31019,31033,31041,31047,31081,31089,31111,31119,31131,31137,31151,31163,31177,31179,31185,32005,32017,32023,32027,32029,32510,33007,34017,34021,34037,35003,35005,35006,35009,35013,35017,35027,35035,35039,35043,35047,35049,35057,36025,36033,36043,36045,36047,36049,36051,36053,36067,36091,36097,36101,36103,36113,36115,36123,37005,37009,37011,37023,37029,37075,37111,37113,37115,37117,37123,37131,37137,37151,37159,37165,37189,37193,38021,38043,38055,38071,38089,38097,38101,38105,39005,39007,39015,39017,39019,39025,39037,39043,39049,39053,39067,39071,39077,39085,39087,39105,39107,39113,39117,39119,39125,39127,39135,39153,39157,39161,39167,39171,40001,40013,40015,40023,40027,40035,40039,40065,40073,40079,40099,40111,40115,40127,40131,40133,40151,40153,41001,41013,41015,41027,41029,41031,41033,41037,41043,41051,41063,42007,42011,42013,42015,42017,42027,42035,42037,42041,42049,42057,42067,42071,42073,42083,42085,42093,42097,42105,42111,42115,42123,42127,42129,44003,44009,45001,45021,45025,45031,45059,45067,45069,45073,46005,46019,46029,46047,46053,46065,46079,46081,46125,46127,47001,47005,47011,47013,47015,47019,47023,47035,47039,47041,47047,47051,47055,47057,47061,47069,47077,47087,47091,47099,47105,47109,47131,47133,47135,47153,47159,47161,47167,47177,47185,48001,48019,48037,48049,48059,48063,48073,48077,48083,48117,48123,48149,48159,48161,48165,48189,48217,48221,48233,48237,48241,48245,48249,48255,48259,48265,48293,48299,48313,48321,48347,48353,48355,48361,48377,48379,48387,48453,48455,48457,48465,48471,48485,48499,49013,49019,49027,49045,50001,50003,50009,50017,50019,50023,50025,50027,51025,51029,51031,51043,51057,51059,51065,51071,51073,51077,51083,51095,51101,51115,51119,51121,51127,51135,51147,51155,51167,51171,51173,51183,51191,51197,51530,51590,51610,51620,51670,51678,51720,51735,51750,51810,51820,51830,53015,53019,53031,53033,53037,53039,53041,53047,53057,53059,53065,53071,53075,54019,54025,54031,54033,54041,54049,54055,54065,54067,54071,54077,54079,54089,54103,54107,55001,55003,55007,55011,55017,55021,55025,55029,55043,55045,55047,55049,55061,55065,55067,55075,55077,55091,55103,55109,55111,55117,55125,55137,56001,56005,56007,56009,56011,56015,56019,56021,56031,56035,56037,56043
]

In [23]:
for TS in range(37, 38):
    print(TS)
    
    model = model3()
            
    train_indices = (raw["istest"]==0) & (raw["dcount"] < TS) & (raw["dcount"] >= 1) & (raw["lastactive"]>ACT_THR) & (raw["lasttarget"]>ABS_THR)
    valid_indices = (raw["istest"]==0) & (raw["dcount"] == TS)
    model.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, 'target'].clip(-0.0043, 0.0045)
    )

    ypred = model.predict(raw.loc[valid_indices, features])
    raw.loc[valid_indices, 'k'] = ypred + 1
    raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

    # Validate
    lastval = raw.loc[raw["dcount"]==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
    dt = raw.loc[raw["dcount"]==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']
    
    df = raw.loc[raw["dcount"]==(TS+1), ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
    df['pred'] = df['cfips'].map(dt)
    df['lastval'] = df['cfips'].map(lastval)
    
    df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
    df.loc[df['lastval']<=ABS_THR, 'pred'] = df.loc[df['lastval']<=ABS_THR, 'lastval']
    df.loc[df['state'].isin(blacklist), 'pred'] = df.loc[df['state'].isin(blacklist), 'lastval']
    df.loc[df['cfips'].isin(blacklistcfips), 'pred'] = df.loc[df['cfips'].isin(blacklistcfips), 'lastval']
    raw.loc[raw["dcount"]==(TS+1), 'ypred'] = df['pred'].values
    raw.loc[raw["dcount"]==(TS+1), 'ypred_last'] = df['lastval'].values
    
    print(f'TS: {TS}')
    print('Last Value SMAPE:', smape(df['microbusiness_density'], df['lastval']) )
    print('SMAPE:', smape(df['microbusiness_density'], df['pred']))
    print()

37


C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not 

TS: 37
Last Value SMAPE: 1.101119095956366
SMAPE: 1.0605959432371874



In [24]:
ind = (raw.dcount>=30)&(raw.dcount<=38)
print('SMAPE:', smape( raw.loc[ind, 'microbusiness_density'], raw.loc[ind, 'ypred']))
print('Last Value SMAPE:', smape(raw.loc[ind, 'microbusiness_density'], raw.loc[ind, 'ypred_last']))

SMAPE: nan
Last Value SMAPE: nan


In [25]:
raw['error'] = vsmape(raw['microbusiness_density'], raw['ypred'])
raw['error_last'] = vsmape(raw['microbusiness_density'], raw['ypred_last'])
raw.loc[(raw.dcount==30), ['microbusiness_density', 'ypred', 'error', 'error_last'] ]

,microbusiness_density,ypred,error,error_last
30,3.334431,NaN,NaN,NaN
77,7.823300,NaN,NaN,NaN
124,1.206827,NaN,NaN,NaN
171,1.236650,NaN,NaN,NaN
218,1.777708,NaN,NaN,NaN
...,...,...,...,...
147140,2.892446,NaN,NaN,NaN
147187,25.438322,NaN,NaN,NaN
147234,3.954258,NaN,NaN,NaN
147281,3.027295,NaN,NaN,NaN


In [26]:
dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38) ].groupby(['state'])['error', 'error_last'].mean()
dt['hit'] = dt['error'] - dt['error_last']
dt = dt.sort_values('hit', ascending=True)
dt.shape

C:\Users\gmrfy\AppData\Local\Temp\ipykernel_1740\2697617936.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38) ].groupby(['state'])['error', 'error_last'].mean()


(51, 3)

In [27]:
dt

,error,error_last,hit
state,,,
District of Columbia,0.059517,0.394946,-0.335429
Delaware,1.083889,1.298054,-0.214165
Massachusetts,0.306108,0.446152,-0.140043
Hawaii,0.209851,0.339925,-0.130074
New Hampshire,0.556332,0.671661,-0.115329
California,0.755002,0.854254,-0.099252
Florida,0.625905,0.723157,-0.097252
New Jersey,0.335024,0.431979,-0.096955
North Carolina,0.654952,0.749359,-0.094408


In [30]:
dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38) ].groupby(['cfips','dcount'])['error', 'error_last'].last()
dt['miss'] = dt['error'] > dt['error_last']
dt = dt.groupby('cfips')['miss'].mean()
dt = dt.loc[dt>=0.50]
dt.shape

C:\Users\gmrfy\AppData\Local\Temp\ipykernel_1740\956906285.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dt = raw.loc[(raw.dcount>=30)&(raw.dcount<=38) ].groupby(['cfips','dcount'])['error', 'error_last'].last()


(0,)

In [31]:
len(dt.index)

0

In [32]:
','.join([str(i) for i in dt.index])

''

In [33]:
for d in dt.index[:10]:
    raw.loc[raw.cfips==d].plot(x='dcount', y=['microbusiness_density', 'ypred'], title=str(d))

In [34]:
raw.iloc[-40:, :16]

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,year,month,dcount,county_i,state_i,mbd_lag_1,dif,target
147305,56045_2020-03-01,56045,Weston County,Wyoming,2020-03-01,1.537087,86.0,0,2020,3,7,45,56,0.011765,0.011765,0.011628
147306,56045_2020-04-01,56045,Weston County,Wyoming,2020-04-01,1.554960,87.0,0,2020,4,8,45,56,0.011628,0.011628,0.034483
147307,56045_2020-05-01,56045,Weston County,Wyoming,2020-05-01,1.608579,90.0,0,2020,5,9,45,56,0.034483,0.034483,0.033333
147308,56045_2020-06-01,56045,Weston County,Wyoming,2020-06-01,1.662198,93.0,0,2020,6,10,45,56,0.033333,0.033333,-0.064516
147309,56045_2020-07-01,56045,Weston County,Wyoming,2020-07-01,1.554960,87.0,0,2020,7,11,45,56,-0.064516,0.064516,0.022989
147310,56045_2020-08-01,56045,Weston County,Wyoming,2020-08-01,1.590706,89.0,0,2020,8,12,45,56,0.022989,0.022989,0.000000
147311,56045_2020-09-01,56045,Weston County,Wyoming,2020-09-01,1.590706,89.0,0,2020,9,13,45,56,0.000000,0.000000,-0.022472
147312,56045_2020-10-01,56045,Weston County,Wyoming,2020-10-01,1.554960,87.0,0,2020,10,14,45,56,-0.022472,0.022472,0.000000
147313,56045_2020-11-01,56045,Weston County,Wyoming,2020-11-01,1.554960,87.0,0,2020,11,15,45,56,0.000000,0.000000,0.011494
147314,56045_2020-12-01,56045,Weston County,Wyoming,2020-12-01,1.572833,88.0,0,2020,12,16,45,56,0.011494,0.011494,0.016450


In [ ]:
blacklist = [
     'North Dakota', 'Iowa', 'Kansas', 'Nebraska', 'South Dakota','New Mexico', 'Alaska', 'Vermont'
]

In [35]:
TS = 38
print(TS)

model0 = model3()
model1 = model3()

train_indices = (raw["istest"]==0) & (raw["dcount"]  < TS) & (raw["dcount"] >= 1) & (raw["lastactive"]>ACT_THR)  & (raw["lasttarget"]>ABS_THR) 
valid_indices = (raw["dcount"] == TS)
model0.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, 'target'].clip(-0.0044, 0.0046),
)
model1.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, 'target'].clip(-0.0044, 0.0046),
)

38


C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\gmrfy\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not 

VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.5,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                                            Pipeline(steps=[('imputer',
                                                                             IterativeImputer(max_iter=20,
                                                                                              n_nearest_features=2)),
                                                                            ('rfr',
                                                                             RandomForestRegressor())])),
                                                           ('ets',
                                                            Pipeline(steps=[('imputer',
                                                                             IterativeImputer(max_iter=20,
                                                                                              n_nearest_features=2)),
                                                                            ('ets',
                                                                             ExtraTreesRegressor())])),
                                                           ('BayesianRidge',
                                                            Pipeline(steps=[('imputer',
                                                                             IterativeImputer(max_iter=20,
                                                                                              n_nearest_features=2)),
                                                                            ('BayesianRidge',
                                                                             BayesianRidge())]))]))])

In [36]:
ypred = (model0.predict(raw.loc[valid_indices, features]) + model1.predict(raw.loc[valid_indices, features]))/2
raw.loc[valid_indices, 'k'] = ypred + 1.
raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

In [37]:
# Validate
lastval = raw.loc[raw["dcount"]==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
dt = raw.loc[raw["dcount"]==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']

In [38]:
df = raw.loc[raw["dcount"]==(TS+1), ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
df['pred'] = df['cfips'].map(dt)
df['lastval'] = df['cfips'].map(lastval)

In [41]:
df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
df.loc[df['lastval']<=ABS_THR, 'pred'] = df.loc[df['lastval']<=ABS_THR, 'lastval']
df.loc[df['state'].isin(blacklist), 'pred'] = df.loc[df['state'].isin(blacklist), 'lastval']
df.loc[df['cfips'].isin(blacklistcfips), 'pred'] = df.loc[df['cfips'].isin(blacklistcfips), 'lastval']
raw.loc[raw["dcount"]==(TS+1), 'ypred'] = df['pred'].values
raw.loc[raw["dcount"]==(TS+1), 'ypred_last'] = df['lastval'].values

In [42]:
df.head()

,cfips,microbusiness_density,state,lastactive,mbd_lag_1,pred,lastval
0,1001,NaN,Alabama,1472.0,NaN,3.465962,3.463856
1,1003,NaN,Alabama,14320.0,NaN,8.365426,8.359798
2,1005,NaN,Alabama,244.0,NaN,1.233059,1.232074
3,1007,NaN,Alabama,229.0,NaN,1.287226,1.287240
4,1009,NaN,Alabama,813.0,NaN,1.832240,1.831783


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3135 entries, 0 to 3134
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cfips                  3135 non-null   int64  
 1   microbusiness_density  0 non-null      float64
 2   state                  3135 non-null   object 
 3   lastactive             3135 non-null   float64
 4   mbd_lag_1              2 non-null      float64
 5   pred                   3135 non-null   float64
 6   lastval                3135 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 171.6+ KB


In [44]:
raw[['cfips','microbusiness_density','dcount','ypred','ypred_last','k']].tail(15)

,cfips,microbusiness_density,dcount,ypred,ypred_last,k
147330,56045,1.767542,32,NaN,NaN,1.000000
147331,56045,1.803249,33,NaN,NaN,1.000000
147332,56045,1.803249,34,NaN,NaN,1.000000
147333,56045,1.803249,35,NaN,NaN,1.000000
147334,56045,1.785395,36,NaN,NaN,1.000000
147335,56045,1.785395,37,NaN,NaN,1.786569
147336,56045,1.785395,38,1.785395,1.785395,1.786256
147337,56045,NaN,39,1.785395,1.785395,1.000000
147338,56045,NaN,40,NaN,NaN,1.000000
147339,56045,NaN,41,NaN,NaN,1.000000


In [45]:
raw.loc[raw['cfips']==28055, 'microbusiness_density'] = 0
raw.loc[raw['cfips']==48269, 'microbusiness_density'] = 1.762115

In [46]:
dt = raw.loc[raw["dcount"]==39, ['cfips', 'ypred']].set_index('cfips').to_dict()['ypred']
test = raw.loc[raw["istest"]==1, ['row_id', 'cfips','microbusiness_density']].copy()
test['microbusiness_density'] = test['cfips'].map(dt)

In [50]:
test = test[['row_id','microbusiness_density']]

In [53]:
print(test)

                  row_id  microbusiness_density
39       1001_2022-11-01               3.465962
40       1001_2022-12-01               3.465962
41       1001_2023-01-01               3.465962
42       1001_2023-02-01               3.465962
43       1001_2023-03-01               3.465962
...                  ...                    ...
147340  56045_2023-02-01               1.785395
147341  56045_2023-03-01               1.785395
147342  56045_2023-04-01               1.785395
147343  56045_2023-05-01               1.785395
147344  56045_2023-06-01               1.785395

[25080 rows x 2 columns]


In [54]:
test.to_csv('submission_model3.csv', index=False)   # model3 : 1.0853(raw0)
test.head()

,row_id,microbusiness_density
39,1001_2022-11-01,3.465962
40,1001_2022-12-01,3.465962
41,1001_2023-01-01,3.465962
42,1001_2023-02-01,3.465962
43,1001_2023-03-01,3.465962
